In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [2]:
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

### Airline_Delay&Cancellation Dataset

In [3]:
df1 = spark.read.csv("gs://airline_bigdata/Data/2009.csv", header=True)
df2 = spark.read.csv("gs://airline_bigdata/Data/2010.csv", header=True)
df3 = spark.read.csv("gs://airline_bigdata/Data/2011.csv", header=True)
df4 = spark.read.csv("gs://airline_bigdata/Data/2012.csv", header=True)
df5 = spark.read.csv("gs://airline_bigdata/Data/2013.csv", header=True)
df6 = spark.read.csv("gs://airline_bigdata/Data/2014.csv", header=True)
df7 = spark.read.csv("gs://airline_bigdata/Data/2015.csv", header=True)
df8 = spark.read.csv("gs://airline_bigdata/Data/2016.csv", header=True)
df9 = spark.read.csv("gs://airline_bigdata/Data/2017.csv", header=True)
df10 = spark.read.csv("gs://airline_bigdata/Data/2018.csv",header=True)

In [4]:
# sum of rows of 10 datasets
df1.count()+ df2.count()+ df3.count()+ df4.count()+ df5.count()+ df6.count()+ df7.count()+ df8.count()+ df9.count()+ df10.count()

61556964

In [5]:
# merge 10 datasets into 1 dataset
df = [df1,df2, df3, df4, df5, df6, df7, df8, df9, df10]
df = reduce(DataFrame.unionAll, df)

In [6]:
# count num of rows after merging 
df.count()

61556964

In [7]:
# Identify columns with missing values
df_missing = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df_missing

22/03/11 22:53:20 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|      0| 

In [8]:
# drop unecessnary columns and remove null rows
df = df.drop('Unnamed: 27', 'LATE_AIRCRAFT_DELAY', 'SECURITY_DELAY', 'NAS_DELAY', 'WEATHER_DELAY', 'CARRIER_DELAY', 'CANCELLATION_CODE')
df = df.na.drop()

In [9]:
# check missing values again, no missing values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|      0|         0|                0|     0|   0|           0|       0|        0|       0|         0|        0|      0|           0|       0|        0|        0|       0|               0|                  0|       0|       0|
+-------+----------+-----------------+------+----+------------+--------+---------+--------+-

In [10]:
# num rows after cleaning
df.count()

60431020

In [11]:
df.show(5)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2009-01-01|        XE|             1204|   DCA| EWR|        1100|  1058.0|     -2.0|    18.0|    1116.0|   1158.0|    8.0|        1202|  1206.0|      4.0|      0.0|     0.0|            62.0|               68.0|    42.0|   199.0|
|2009-01-01|        XE|             1206|   EWR| IAD|        1510|  1509.0|     

### Airline dat. Dataset

In [12]:
df_air = spark.read.csv("gs://airline_bigdata/Data/airports.dat.txt", header = False)

In [13]:
df_air.count()

7698

In [14]:
df_air = df_air.drop("_c0", "_c5", "_c8","_c9","_c10","_c11","_c12", "_c13")
df_air

_c1,_c2,_c3,_c4,_c6,_c7
Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081689834590001,145.391998291
Madang Airport,Madang,Papua New Guinea,MAG,-5.20707988739,145.789001465
Mount Hagen Kagam...,Mount Hagen,Papua New Guinea,HGU,-5.826789855957031,144.29600524902344
Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977
Port Moresby Jack...,Port Moresby,Papua New Guinea,POM,-9.443380355834961,147.22000122070312
Wewak Internation...,Wewak,Papua New Guinea,WWK,-3.58383011818,143.669006348
Narsarsuaq Airport,Narssarssuaq,Greenland,UAK,61.1604995728,-45.4259986877
Godthaab / Nuuk A...,Godthaab,Greenland,GOH,64.19090271,-51.6781005859
Kangerlussuaq Air...,Sondrestrom,Greenland,SFJ,67.0122218992,-50.7116031647
Thule Air Base,Thule,Greenland,THU,76.5311965942,-68.7032012939


In [15]:
# Subselect 
#df_air.select("name.firstname","name.lastname").show(truncate=False)
df_air = df_air.filter( df_air._c3 == "United States")

In [16]:
# count row number after subselection
df_air.count()

1512

In [17]:
df_air = df_air.selectExpr("_c1 as Airport_name", "_c2 as City", "_c3 as Country", "_c4 as Airport_Code", "_c6 as Latitude", "_c7 as Longtitude")
df_air.printSchema()

root
 |-- Airport_name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)



In [18]:
df_air.show(5)

+--------------------+---------------+-------------+------------+------------------+-------------------+
|        Airport_name|           City|      Country|Airport_Code|          Latitude|         Longtitude|
+--------------------+---------------+-------------+------------+------------------+-------------------+
|Barter Island LRR...|  Barter Island|United States|         BTI|     70.1340026855|     -143.582000732|
|Wainwright Air St...|Fort Wainwright|United States|          \N|       70.61340332|       -159.8600006|
|Cape Lisburne LRR...|  Cape Lisburne|United States|         LUR|       68.87509918|       -166.1100006|
|Point Lay LRRS Ai...|      Point Lay|United States|         PIZ|       69.73290253|       -163.0050049|
|Hilo Internationa...|           Hilo|United States|         ITO|19.721399307250977|-155.04800415039062|
+--------------------+---------------+-------------+------------+------------------+-------------------+
only showing top 5 rows



In [19]:
#check missing values
df_air.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_air.columns]).show()

+------------+----+-------+------------+--------+----------+
|Airport_name|City|Country|Airport_Code|Latitude|Longtitude|
+------------+----+-------+------------+--------+----------+
|           0|   0|      0|           0|       0|         0|
+------------+----+-------+------------+--------+----------+



In [20]:
# inner join df and df_air dataset on Airport_Code and ORIGIN
df_final = df.join(df_air, df.ORIGIN == df_air.Airport_Code,"inner")

In [21]:
# rename columns
df_final = df_final.withColumnRenamed("Airport_name", "OR_AIRPORT_NAME ")
df_final = df_final.withColumnRenamed("City", "OR_CITY")
df_final = df_final.withColumnRenamed("Country", "OR_COUNTRY")
df_final = df_final.withColumnRenamed("Airport_Code","OR_AIRPORT_CODE")
df_final = df_final.withColumnRenamed("Latitude", "OR_LATITUDE")
df_final = df_final.withColumnRenamed("Longtitude", "OR_LONGTITUDE")
df_final.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- OR_AIRPORT_NAME : string (nullable = true)
 |-- OR_CITY: string (nullable = true)
 |-- OR_COUNTRY: string (nullabl

In [22]:
# inner join df_final and df_air dataset on Airport_Code and DEST
df_final = df_final.join(df_air, df_final.DEST == df_air.Airport_Code,"inner")

In [23]:
# rename columns
df_final = df_final.withColumnRenamed("Airport_name", "DES_AIRPORT_NAME ")
df_final = df_final.withColumnRenamed("City", "DES_CITY")
df_final = df_final.withColumnRenamed("Country", "DES_COUNTRY")
df_final = df_final.withColumnRenamed("Airport_Code","DES_AIRPORT_CODE")
df_final = df_final.withColumnRenamed("Latitude", "DES_LATITUDE")
df_final = df_final.withColumnRenamed("Longtitude", "DES_LONGTITUDE")
df_final.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- OR_AIRPORT_NAME : string (nullable = true)
 |-- OR_CITY: string (nullable = true)
 |-- OR_COUNTRY: string (nullabl

In [24]:
# Count num of rows of cleaned dataset
df_final.count()

59820609

### Export Data for EDA

In [ ]:
# export cleaned dataset to csv
df_final.coalesce(1).write.csv("gs://airline_bigdata/Data/flights_clean", header= True)

### Export Data for Classification 

In [ ]:
df = df.select('FL_DATE','OP_CARRIER', 'ORIGIN', 'CRS_DEP_TIME', 'CRS_ELAPSED_TIME', 'CANCELLED')
df.count()

In [ ]:
df.coalesce(1).write.csv("gs://airline_bigdata/Data/logistics_clean", header= True)

### Export Data for Regression

#### Data Processing 

In [ ]:
# change date format
df = df.withColumn("FL_DATE",to_date(col("FL_DATE"),"yyyy-MM-dd"))
# calculate average delay
df = df.withColumn('AVG_DELAY',((col("DEP_DELAY") + col("ARR_DELAY")) / lit(2)))
# change date to season
df = df.withColumn("SEASON", \
   when((month(df.FL_DATE) >= 3) & (month(df.FL_DATE) <= 5), lit("Spring")) \
     .when((month(df.FL_DATE) >= 6) & (month(df.FL_DATE) <= 8), lit("Summer")) \
.when((month(df.FL_DATE) >= 9) & (month(df.FL_DATE) <= 11), lit("Fall")) \
     .otherwise(lit("Winter")) \
  )
# select columns for regression analysis
df_reg = df.select("AVG_DELAY", "OP_CARRIER","ORIGIN", "CRS_DEP_TIME", "TAXI_OUT", "TAXI_IN", "CRS_ELAPSED_TIME", "SEASON")

In [ ]:
df_reg = df_reg.withColumn("label", df_reg.AVG_DELAY)
df_reg = df_reg.drop("AVG_DELAY")
df_reg

In [ ]:
df_reg.coalesce(1).write.csv("gs://airline_bigdata/Data/regression_clean", header= True)